In [2]:
import numpy as np

In [3]:
with open('input.txt', 'r') as f:
    text = f.read()
print(f'Length of text: {len(text)}')


Length of text: 1115394


In [4]:
print(text[:16])

First Citizen:
B


In [5]:
#before beginning, let us fist see kaun kaunse characters use ho rahe hain
chars = set(text) #unique characters choose ho gaye apne aap, unordered
chars = sorted(list(set(text))) #list of set -> we get an ordering, an arbitrary ordering tho. Then sorted makes it a particular ordering
chars
print(''.join(chars))
vocab_size = len(chars) #these characters are our vocabulary, we'll make new words etc from these
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
#create a mapping from characters to integers
#tokenize: convert the string of chars to some sequence of integers acc to some method

#one schema for tokenizing is creating a simple look-up table
stoi = {ch:i for i,ch in enumerate(chars)} #string to integer
itos = {i:ch for i, ch in enumerate(chars)}



In [7]:
stoi['A']

13

In [8]:
[stoi[c] for c in "Hello"]

[20, 43, 50, 50, 53]

In [9]:
[itos[c] for c in [20, 43, 50, 50, 53]]

['H', 'e', 'l', 'l', 'o']

In [10]:
encoder = lambda s: [stoi[c] for c in s] #Takes in s and then does [..] on s 
decoder = lambda s: [itos[c] for c in s]
enc = encoder('Hello')
dec = decoder(enc)
print(enc)
print(''.join(dec))
def decode(x):
    return ''.join(decoder(x))

decode(encoder('Hello'))

[20, 43, 50, 50, 53]
Hello


'Hello'

In [11]:
#Let us encode and tokenize our dataset
data = encoder(text)
import torch 
data = torch.tensor(encoder(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
#Split data into train adn validation test
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
#We do not feed the whole training data into the neural networks, we work with chunks of the data
#we sample random chunks, some maximum length, aka block size
#we call this chunks as context window, or block size
block_size = 8
train_data[:block_size+1] #here are 9 characters, with 8 examples to train on
#18 ke bad 47 aata hai
#18 & 47 ke bad 56
#18,47,56, ke bad 57 etc and so on


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size] #block size characters
y = train_data[1:block_size+1] #next block_size characters, since it is offset by 1
for t in range(block_size):
    context = x[:t+1] #input is the chars upto and including t
    target = y[t] #target is the t-th character in target array y
    print(f"when input is {context} the target: {target}")


when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [15]:
torch.manual_seed(1337)
#creating batches of the data so that the GPUs are busy and GPUs can train them independently
batch_size = 4 #number of indepndent sequences to be trained in parallel every fwd and backward passs of the transformer
block_size = 8 #what is max context length for predictions?

def get_batch(split):
    #generate a small batch of data of inputs x an y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,)) #smple a Random location in the whole dataset, pure dataset (from 0 to len(data)-blocksize) me se ek random sample lena hai, 'batch_size'=4 jitne random nos.
    x = torch.stack([data[i:i+block_size] for i in ix]) #.stack(list, dim=0 default) - stack them in rows of 4 rows (batch size) x 8 size ka tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

# ix, _, _ = get_batch('train')
# [data[i:i+block_size] for i in ix]
# [data[i] for i in ix]
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension or block/context window dimention
        context = xb[b, :t+1] #b-th batch item lo, us itme ke 0 se t+1 tk block chars lo. Input comes from x array
        target = yb[b, t] #bth batch item lo, aur us itme ka sirf tth char lo. Target comes from y
        print(f"when input is {context} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is tensor([24]) the target: 43
when input is tensor([24, 43]) the target: 58
when input is tensor([24, 43, 58]) the target: 5
when input is tensor([24, 43, 58,  5]) the target: 57
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39
when input is tensor([44]) the target: 53
when input is tensor([44, 53]) the target: 56
when input is tensor([44, 53, 56]) the target: 1
when input is tenso

In [16]:
#Now we feed the train data to the simplest language model neural network - i.e. bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) 
        #we create a token embedding table of vocabsize x vocabsize

    def forward(self, idx, targets = None):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #(B,T,C)
        #when we pass an index, i.e. xb of size 4x8, then 
        #every single integer of our xb will refer to the embedding table
        #and plucks out a row of that embedding table corresponding to its index
        #ex: we have 
        # xb = > torch.Size([4, 8])
        # tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        #         [44, 53, 56,  1, 58, 46, 39, 58],
        #         [52, 58,  1, 58, 46, 39, 58,  1],
        #         [25, 17, 27, 10,  0, 21,  1, 54]])
        #so, 24 will pluck out 24th row, 
        # 43 will pluck out 43rd row and so on
        #and then pytorch arranges them into B,T,C - batch, time, channel tensor
        #batch = 4, time = 8, channel = 65 (vocab size), C is also the 'classes'. Basically, the classes into which we're classifying
        #and we interpret them as logits, ie scores for next character in the sequence
        #-ve log liklihood loss aka cross entropy

        if targets is None: 
            loss = None
        else:
            B,T,C = logits.shape
            # print(B,T,C)
            logits = logits.view(B*T, C) #pytorch wants 'C' as 2nd dimension, so we squash/combine the first 2 dims into one dim and make C as 2nd dim
            # print(logits.shape)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is (B,T) array of the B no. of current context-s (of length T)
        #job of generate is to extend the (B,T) to (B, T+1) to (B, T+2) and so on
        #i.e. for all rows in a Batch (i.e. B =4) the columns inc from T to T+1, T+1 is then fed back to generate (T+1)+1 = T+2 and this goes on till the number of max_tookens that we want
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #Since this is a bigram model, so focus only on the last time step, pluck out the last step as it will give the prediction for next step. Even though we're feeding all the history into the model, only last character is being used. History is not being used in bigram model rn.
            logits = logits[:, -1, :] # logits original size: (B,T,C), The last token's logits for each sequence in the batch, .'. becomes (B,C)
            #apply softmax to get probablities
            probs = F.softmax(logits, dim=-1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B,1) #For each batch row, we've 1 prediction for what comes next
            #append sampled indec to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) i.e. concatenated along the 1st dim i.e. T, 0th dim is B here i.e. rows
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) #forward() is called -> forward(self, xb, yb) .'. idx = xb 
print(logits.shape) #out.shape
print(loss) #-log(1/65) ~ 4.17

#Let's make a little 1x1 tensor of 0, i.e. a newline character (or space?) and see 
idx = torch.zeros((1,1), dtype = torch.long) #datatype is int
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist())) #generate returns (B, max_token_size) so, just pluck the 0th batch row  -> convert it to python list  


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
#Let's now train
#let's create a Pytorch optimiser
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for steps in range(10000):
    #sample a batch of data
    xb, yb = get_batch('train')
    #evaluate the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True) #zero the gradients from the previous steps
    loss.backward() #getting the gradients for all the parameters
    optimizer.step() #using those gradients to update the parameters
print(loss.item())


2.4212486743927


In [18]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),max_new_tokens=400)[0].tolist()))


KIZ!

we?
W$d he k
MIV;


IETyoZY st:


Gheay Yatondgrende'TEQUNathinin: ans nd s

OREre.
Qhour im, sof bn I's temrpe thithattersuravWhefe h antr forgg m walos nt kit mo RLORZ'd IFknyor!
Sindis prewhous oodod.
wouprebe bll:CUNTHUSt

Thleaix;
Malesheais whe; lino.
ALI;
CXdaimboth,
batly, tyous w F nig har-
ASerin livin ad:
GRveitho wond ve ack'
3'd gat held inghe beromUCOFFoce ?Y's.
Pat fe, healcof


## The mathematical trick in self-attention

In [19]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch, time, channels (embedding length)
x = torch.randn(B,T,C)
x.shape



torch.Size([4, 8, 2])

In [20]:
#we want to have some sort of communication among the tokens now
#we want to have current token to take in average of feature vectors of previous tokens
#We want x[b,t] = mean_{i<=t} x[b,i]
#x-bag-of-words, 'bag-of-words' a term people use when they wanna refer to the average of somethings
xbow = torch.zeros((B,T,C)) #initialize the average to 0

for b in range(B):
    # print(b)
    for t in range(T):
        xprev = x[b,:t+1] #(t,C), #everythin in the current batch dimension b, and upto and including t-th dimension i.e. (1,t,C) or (t,C)
        xbow[b,t] = torch.mean(xprev, 0) #average along the row (axis = 0), i.e. collapsing the time, so we get (C)
        # print(xprev)


In [21]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [22]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [23]:
#This was inefficient
#Let's use mathematical trick
#matrix multiplicaiton

#let's use toy example:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a@b  #cross or simply matrix multiplicaiton
print(f"a={a}")
print('--')
print(f"b={b}")
print('--')
print(f"c={c}")

a=tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [24]:
#THE TRICK:
#Use a triangular matrix to do the 'sliding window' mean
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
#sum along the cols (axis = 1), #kepdim = True, so that o/p dim retains column dim, i.e. o/p dim is [3,1] (had keep dim = False, o/p dim would've been [3]) .
#why keep dim=True? 'cause o/p of shape [3,1] on broadcasting can become [3,3], i.e. columns are just copied while broadcasting and then division happens
#which is not possible with o/p of shape [3]
a = a/torch.sum(a, 1, keepdim=True) 
b = torch.randint(0,10,(3,2)).float()
c = a@b  #cross or simply matrix multiplicaiton
print(f"a={a}")
print('--')
print(f"b={b}")
print('--')
print(f"c={c}")
#1st row (with 1,0,0) x 1st col and 2nd col = just 1st row and just 1st row
#2nd row (w. 1,1,0) x 1st col and 2nd col = 1st+2nd row and 1st+2nd row
#3rd row (w. 1,1,1) x 1st col and 2nd col = 1st+2nd+3rd row and 1st+2nd+3rd row
#i.e. the "sliding window mean", or "accumulative sum" can be easily done using just matrix multiplication


a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [25]:
#version 2:
#Let's implement this in our accumulating sum/mean 
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x 
# (T,T) @ (B,T,C) 
# pytorch makes it:
# (B,T,T) @ (B,T,C)
# then we have, (B,T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(xbow, xbow2, rtol=1e-5, atol=1e-5)

#What is wei? it is nothing but weights.


True

In [26]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [27]:
#now, in elementwise softmax, it exponentiates the elements, 
#then divides the row with the sum of the row
#softmax is like normalisation. #wei here is exact same as wei in version2
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [28]:
#version 3:
#adding softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) 
xbow3 = wei@x
torch.allclose(xbow, xbow3, rtol=1e-5, atol=1e-8)

#why we will use version 3 in self attention:
# Weights begin with 0. We can think of this as 'intereaction strength' or 'affinity' 
#tells us how much the tokens of the past that we wanna aggregate/average out
#wei = torch.zeros((T,T))
#Then, wei.masked_fill(tril == 0, float('-inf')) line says, tokens from the past, cannot communicate.
#we set them to -inf, i.e. we will not aggregate anything from those tokens
#then it goes thru softmax, and aggregation "wei@x" thru matrix multiplication



False

In [ ]:
#version 4: self-attention!
from httpx import head


torch.manual_seed(1337)
B,T,C = 4,8,32 #batch, time/contex length, channels(dimension of identity embedding)
x = torch.randn(B, T, C)

#the following creates a "unirofm" distr affinities/association of a token and token previous it, which then lets us do simple average
#but instead of this bland/uninformative averaging, we want an informed probab distribution of the previous tokens, representing affinities!
#jump->
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) 
out = wei@x

#->jump
#1. Self-Attention
#Now, every single token, at each position, emits 2 vectors: key and query
#query: what is the current token looking for
#key: what do I contain (content and position)?
#now, the way we get affinities bw tokens is: that we simple take a dot product bw keys and queries
#My Query dot product with all the keys of all the other tokens. Jiske sath jyada value aayi, uske sath affinity jyada
#and this affinity is nothing but wei , or weights

#lets see a single head perform self-attention
head_size = 16
key = nn.Linear(C,head_size, bias=False) #sans bias, it just performs matrix multiplicaiton (xW') output=xW'
query = nn.Linear(C, head_size, bias=False)
k = key(x) #x:(B,T,C), key=nn.Linear(C,16) .'. o/p: B,T,16
q = query(x) #x:(B,T,C), key=nn.Linear(C,16) .'.  o/p: B,T,16
#abhi every token at each position ne bus apne-apne query and keys bana liye hain, no cmmunicaiton has happened till now
#Now, communication will happen ↓
wei = q @ k.transpose(-2,-1) #(B,T,16) @ (B,16,T) ---> (B,T,T) #ye TxT hi 'attention scores hain'
#then just do the same thing as above
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) 
out = wei@x
out.shape

#2. Self.Attention
#Now, we do one more thing: 'value'
#We do not aggergate the affinity-scores of previous token i.e. we wont do wei@x now
#instead, we aggreate the value vector instead of raw x
head_size = 16
key = nn.Linear(C,head_size, bias=False) #just a head_size x C matrix #16x32 # 'here's what I (a token) have'
query = nn.Linear(C, head_size, bias=False) # head_size x C matrix #16x32    # 'here's what I'm (a token) interested in'
value = nn.Linear(C, head_size, bias=False) # head_size x C matrix #16x32    # and, if you find me interesting (affinity score) 'here's what I'll communicate to you'
k = key(x) #(4,8,32)x(16,32).T = k:(4,8,16)
q = query(x) #(4,8,32)x(16,32).T = q:(4,8,16)

wei = q@ k.transpose(-2,-1)
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) 

v = value(x) #x:(B,T,C), key=nn.Linear(C,16) .'. o/p: B,T,16 #(4,8,32)x(16,32).T = q:(4,8,16)
out = wei @ v #we aggregate v instead of raw x
#out = wei @ x  #x is kinda like private information to this token
out.shape #(4,8,16)  output is 16 dimentional instead of 32, '.' head_size is 16, based on it, value is 16x32

torch.Size([4, 8, 16])

In [42]:
wei.shape

torch.Size([4, 8, 8])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. These 'nodes' have no idea where they're placed in space. This is why we need to positionally encode tokens. Unlike convolutional layer where there is notion of spatial information.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other. So, in graph analogy, we atually have 4 seprate pools (if batch_size is 4) of 8 nodes (context/time length)
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. The block we have here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [46]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5
wei = q @ k.transpose(-2, -1) #* head_size**-0.5

In [47]:
k.var()

tensor(1.0966)

In [48]:
q.var()

tensor(0.9416)

In [49]:
wei.var()

tensor(16.1036)

In [ ]:
#Softmax tries to become like 1-hot vector if the i/p vector has extream-er values
#vector with values close to 0 will give a 'diffused' vector like ↓
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

In [ ]:
#vector with extremer values tend to converge to 1-Hot vector
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot
#So, ye na ho, that's why we do 'scaled attention'. " "Scaled" attention additional divides `wei` by 1/sqrt(head_size)."
#scaling is used to control the variance at initialisation

In [50]:
import sys
print(sys.executable)

c:\Users\himan\miniconda3\python.exe


In [ ]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    def __call__(self, x):
    # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # batch mean  #axis=0: we normalise the columns (=batch norm), axis=1: we normalise the rows (=layer norm)
        xvar = x.var(1, keepdim=True) # batch variance 
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32,100) #batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [56]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [55]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))